In [11]:
import pandas as pd
import os

def generate_track_pages(csv_path, output_dir='output'):
  
    try:
       
        df = pd.read_csv(csv_path)
        
        
        required_columns = ['title', 'abstract', 'authors', 'track', 'invited','transaction']
        if not all(col in df.columns for col in required_columns):
            missing = [col for col in required_columns if col not in df.columns]
            raise ValueError(f"CSV缺少必要列：{missing}")

        os.makedirs(output_dir, exist_ok=True)

        
        track_files = []
        for track_name, group in df.groupby('track'):
            
            safe_name = sanitize_filename(track_name)
            filename = os.path.join(output_dir, f"{safe_name}.html")
            
            
            html_content = generate_single_track_html(track_name, group)
            
           
            with open(filename, 'w', encoding='utf-8') as f:
                f.write(html_content)
            track_files.append((track_name, filename))

        
        generate_index_page(track_files, output_dir)
        
        print(f"Success, at{os.path.abspath(output_dir)}")

    except Exception as e:
        print(f"Fail：{str(e)}")

def sanitize_filename(name):
    
    return "".join([c if c.isalnum() or c in (' ', '_') else '_' for c in name]).strip()

def generate_single_track_html(track_name, df):
    
    papers_html = []
    for _, row in df.iterrows():
        invited_tag = '<span class="invited">[Invited]</span>' if row['invited'] else ''
        papers_html.append(f"""
        <div class="paper">
            <h3 class="title">[<a href={"..\\pdf\\"+row['transaction']+"-final.pdf"}>{row['title']}{invited_tag}</a>]</h3>
            <div class="authors">{row['authors']}</div>
            <div class="abstract">{row['abstract']}</div>
            
        </div>
        """)
    
    return f"""<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <title>{track_name}</title>
    <style>
        body {{ font-family: Arial, sans-serif; max-width: 800px; margin: 0 auto; padding: 20px; }}
        h1 {{ color: #2c3e50; border-bottom: 2px solid #3498db; }}
        .paper {{ margin: 20px 0; padding: 15px; border-left: 4px solid #3498db; }}
        .title {{ font-weight: bold; color: #2c3e50; }}
        .authors {{ color: #7f8c8d; font-size: 0.9em;font-style: italic; }}
        .abstract {{ color: #666; margin-top: 15px; font-size: 0.95em; }}
        .invited {{ color: #27ae60; font-weight: bold; }}
    </style>
</head>
<body>
    <h1>{track_name} </h1>
    {"".join(papers_html)}
    <p><a href="index.html">← Back</a></p>
</body>
</html>"""

def generate_index_page(track_files, output_dir):
    
    links = []
    for name, path in track_files:
        filename = os.path.basename(path)
        links.append(f'<li><a href="{filename}">{name}</a></li>')
    
    index_html = f"""<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <title>Tracks</title>
    <style>
        body {{ font-family: Arial, sans-serif; max-width: 600px; margin: 0 auto; padding: 20px; }}
        h1 {{ text-align: center; color: #2c3e50; }}
        ul {{ list-style: none; padding: 0; }}
        li {{ margin: 10px 0; padding: 10px; background: #f8f9fa; border-radius: 4px; }}
        a {{ color: #3498db; text-decoration: none; }}
        a:hover {{ text-decoration: underline; }}
    </style>
</head>
<body>
    <h1>EDTM 2025 Papers</h1>
    <ul>{"".join(links)}</ul>
</body>
</html>"""

    with open(os.path.join(output_dir, 'index.html'), 'w', encoding='utf-8') as f:
        f.write(index_html)


if __name__ == "__main__":
    csv_path = "accepted_paper_list.csv"
    generate_track_pages(csv_path)


Success, ate:\The University Of Hong Kong\CANLab - WEN, Bo - WEN, Bo\EDTM\EDTM_proceeding\output


In [ ]:
import pandas as pd
from pathlib import Path

TRACK_ORDER = [
    "01 Materials",
    "02 Process, Tools, Yield, and Manufacturing",
    "03 Advanced Semiconductor (Logic) Devices",
    "04 Memory Technologies",
    "05 Photonics, Imaging and Display",
    "06 Wide-Bandgap Power and RF Devices",
    "07 Modelling and Simulation",
    "08 Reliability and testing",
    "09 Packaging and Heterogenous Integration",
    "10 Sensor, MEMS, Bio-Electronics",
    "11 Flexible and Wearable Electronics", 
    "12 Nanotechnologies",
    "13 Disruptive Technologies (Metaverse, Neuromorphic Computing, Quantum Computing)"
]

def generate_html_days(csv_path):
    
    df = pd.read_csv(csv_path)
    df['Day'] = df['TimeIndex'].map({0:'Day1',1:'Day1',2:'Day2',3:'Day2',4:'Day3',5:'Day3'})
    
    
    for day in ['Day1', 'Day2', 'Day3']:
        day_df = df[df['Day'] == day]
        html_content = f'''
<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <title>{day} Papers</title>
    <style>
        body {{ font-family: Arial, sans-serif; max-width: 800px; margin: 0 auto; padding: 20px; }}
        h1 {{ color: #2c3e50; border-bottom: 2px solid #3498db; }}
        .paper {{ margin: 20px 0; padding: 15px; border-left: 4px solid #3498db; }}
        .title {{ font-weight: bold; color: #2c3e50; }}
        .authors {{ color: #7f8c8d; font-size: 0.9em; font-style: italic; }}
        .abstract {{ color: #666; margin-top: 15px; font-size: 0.95em; }}
        .invited {{ color: #27ae60; font-weight: bold; }}
    </style>
</head>
<body>
    <h1>{day} Papers</h1>
'''


        for track in TRACK_ORDER:
            track_df = day_df[day_df['track'] == track]
            if not track_df.empty:
                html_content += f'<h2>{track}</h2>\n'
                html_content += '\n'.join([generate_paper(row) for _, row in track_df.iterrows()])

        html_content += '''
</body>
</html>'''
        
        
        with open(f"{day}.html", "w", encoding="utf-8") as f:
            f.write(html_content)

def generate_paper(row):
    invited = '<span class="invited">[Invited]</span> ' if row['invited'] else ''
    return f'''
<div class="paper">
    <div class="title">{invited}{row['title']}</div>
    <div class="authors">{row['authors']}</div>
    <div class="abstract">{row['abstract']}</div>
</div>'''

generate_html_days("program_new.csv")


In [ ]:
import pandas as pd
from jinja2 import Template


csv_file = "poster_new.csv" 
df = pd.read_csv(csv_file)


grouped_data = df.groupby(['day', 'track_poster'])


html_template = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Articles by Date and Track</title>
    <style>
        body { font-family: Arial, sans-serif; max-width: 1000px; margin: 0 auto; padding: 20px; }
        h1 { color: #7F056A; font-family:Cambria, serif; font-style: normal; font-weight: bold; text-decoration: none; font-size: 20pt; border-bottom: 2px solid #3498db;}
        .paper { margin: 20px 0; padding: 15px; border-left: 4px solid #3498db; }
        .title { font-weight: bold; color: #2c3e50; }
        .authors { color: #231F20; font-family:"Book Antiqua", serif; font-style: italic; font-weight: normal; text-decoration: none; font-size: 11pt; }
        .plenary_authors { color: #231F20; font-family:"Book Antiqua", serif; font-style: italic; font-weight: normal; text-decoration: none; font-size: 11pt; }
        .abstract { color: #666; margin-top: 15px; font-size: 0.95em; }
        .invited { color: #27ae60; font-weight: bold; }
        .venue { color: #1F8639; font-family:Cambria, serif; font-style: normal; font-weight: normal; text-decoration: none; font-size: 11pt; }
        .time { color: #7F056A; font-family:Cambria, serif; font-style: normal; font-weight: bold; text-decoration: none; font-size: 18pt; }
        .track { font-weight: bold; }
        .plenary { font-style: italic; }
        .chair { color: #007DAC; font-family:Cambria, serif; font-style: normal; font-weight: bold; text-decoration: none; font-size: 11pt; }
        .block-container {
            padding: 15px;
        }
        .paper-block {
            margin: 15px 0;
            padding: 15px;
            box-shadow: 0 2px 4px rgba(0,0,0,0.1);
        }
    </style>
</head>
<body>
    <h1>Articles Organized by Date and Track</h1>
    {% for date, tracks in data.items() %}
        <div class="block-container">
            <h2 class="time">Date: {{ date }}</h2>
            {% for track, articles in tracks.items() %}
                <div class="paper-block">
                    <div class="track">Track: {{ track }}</div>
                    {% for article in articles %}
                        <div class="paper">
                            <div class="title">{{ article }}</div>
                        </div>
                    {% endfor %}
                </div>
            {% endfor %}
        </div>
    {% endfor %}
</body>
</html>
"""


data = {}
for (date, track), group in grouped_data:
    if date not in data:
        data[date] = {}
    data[date][track] = group['poster_title'].tolist()


template = Template(html_template)
rendered_html = template.render(data=data)


output_file = "poster.html"
with open(output_file, "w", encoding="utf-8") as f:
    f.write(rendered_html)

print(f"HTML file has been generated: {output_file}")

HTML file has been generated: styled_articles_by_date_and_track.html
